In [1]:
import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_da32d840b0c6496da7e4b4ae965e36bf = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_da32d840b0c6496da7e4b4ae965e36bf = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

credentials = {
    'endpoint': endpoint_da32d840b0c6496da7e4b4ae965e36bf,
    'service_id': 'iam-ServiceId-534b61b5-307d-474a-8ba4-deebac0187b5',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'XiBbMTH05VLVP_M-y0DgAdgBNzIrmYT8Mmqr8kX51gzV'
}

configuration_name = 'os_da32d840b0c6496da7e4b4ae965e36bf_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df = spark.read.json(cos.url('medium-sparkify-event-data.json', 'churnspark-donotdelete-pr-pty3wun15oklrj'))
df.take(5)


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210318004239-0000
KERNEL_ID = e9071b9e-0c90-4d65-8dc6-b7f377d802c5


[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist='Afroman', auth='Logged In', firstName='Maverick', gender='M', 

In [2]:
# import libraries
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import *


from pyspark.ml.feature import VectorAssembler, StandardScaler
#from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import datetime

In [3]:
### delete empty userIds
df = df.dropna(how = 'any', subset = ['userId', 'sessionId'])
df = df.filter(df.userId != '')

### delete useless/sensitve information
### i.e. songs, names
df = df.select('auth', 'gender', 'itemInSession',\
     'level', 'location', 'method', 'page', 'registration',\
       'sessionId', 'status', 'ts', 'userAgent', 'userId')

In [5]:
# Users cancelled their paid accounts
flag_cancel_event = F.udf(lambda x: 1 if x == 'Cancellation Confirmation' else 0, IntegerType())
df = df.withColumn('cancel', flag_cancel_event('page'))

windowval = Window.partitionBy('userId').orderBy(F.desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

df = df.withColumn('churn', F.sum('cancel').over(windowval))

# Feature Engineering

In [6]:
### Use Frequency: based on sessions
df_freq = df.groupby('userId').agg({'ts': 'max',\
                                    'registration': 'max',\
                                    'sessionId':'count'})

df_freq = df_freq.withColumn('days', (F.col('max(ts)') - F.col('max(registration)'))/(1000*60*60*24))\
        .withColumn('freq', F.col('count(sessionId)')/F.col('days'))\
        .select('userId', 'freq')

In [7]:
# How many songs do users listen to on average between visiting the home page? 
ifhome = F.udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userId') \
    .orderBy(F.desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userId', 'page', 'ts') \
    .withColumn('homevisit', ifhome(F.col('page'))) \
    .withColumn('period', F.sum('homevisit').over(user_window))

df_songs = cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'})\
    .groupBy('userID') \
    .agg(F.avg('count(period)').alias("Songs"))

In [8]:
# time per session
df_time = df.groupby("userId", "sessionId")\
    .agg((F.max(df.ts)-F.min(df.ts)).alias("duration"))\
    .groupby("userId").agg(F.avg(F.col('duration')).alias("time_session"))


In [9]:
# percentage time in different page type 
time_page_each = df.groupby("userId")\
.pivot('page').agg(F.count('page'))\
.fillna(0)

cols = time_page_each.columns

time_page_total = df.groupby("userId")\
.agg(F.count('page'))

time_page_each = time_page_each.join(time_page_total, ['userId'], how='left')

for col in cols[1:]: # get rid of 'userId' in the cols
    time_page_each =  time_page_each\
        .withColumn(col, F.col(col)/F.col('count(page)'))

df_page = time_page_each.drop('count(page)')

In [10]:
# gender
df_gender = df.select('userId', 'gender').dropDuplicates()

int_gender = F.udf(lambda x: 1 if x == "F" else 0, IntegerType())

df_gender = df_gender.withColumn('gender', int_gender('gender'))

In [11]:
# if ever paid
df_level = df.select('userId', 'level').dropDuplicates()

int_level = F.udf(lambda x: 1 if x == "paid" else 0, IntegerType())

df_level = df_level.withColumn('level', int_level('level'))\
    .groupby('userId').agg(F.max('level').alias('if_paid'))

# Model

In [12]:
# churn
df_churn = df.select('userId', 'churn').dropDuplicates()

Row(userId='100010', churn=1)

In [13]:
### Create full dataset
full_data = df_churn
feature_list = [df_freq, df_songs, df_time, df_page, df_gender, df_level]

for f in feature_list:
    full_data = full_data.join(f, ['userId'], how='left')

In [14]:
# vectorize the features
assembler = VectorAssembler(inputCols=full_data.columns[2:], outputCol="NumFeatures")
full_data = assembler.transform(full_data) 

# standarlize the features
scaler = StandardScaler(inputCol='NumFeatures', outputCol='features', withMean=True, withStd=True)
scalerModel = scaler.fit(full_data)
full_data = scalerModel.transform(full_data)

In [15]:
full_data = full_data.select(F.col('churn').alias("label"), 'features')

In [16]:
train, test = full_data.randomSplit([0.9, 0.1], seed=42)
train.persist()

DataFrame[label: bigint, features: vector]

In [17]:
lr = LogisticRegression(maxIter=10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)
cvModel = crossval.fit(train)

In [18]:
cvModel.avgMetrics

[0.8729759435073445, 0.9240122377102274]

In [19]:
results = cvModel.transform(test)

In [20]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(results.select('label').collect(), results.select('prediction').collect())
print(cm)
tn, fp, fn, tp = cm.ravel()
precision = tp / (tp + fp) 
recall = tp / (tp + fn)
f1 = 2*precision*recall/(precision+recall)
print('precision:', precision)
print('recall:', recall)
print('f1', f1)

[[30  0]
 [ 3  3]]
precision: 1.0
recall: 0.5
f1 0.6666666666666666
